In [38]:
import os
import requests
from bs4 import BeautifulSoup
from openai import OpenAI
from IPython.display import Markdown, display
MODEL = "llama3.2"
openai = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

In [ ]:
# testing the model
msg = "This is Good"
response = openai.chat.completions.create(model = MODEL,
                                          messages= [ {"role": "user", "content": msg}])
print(response)
print(response.choices[0].message.content)

ChatCompletion(id='chatcmpl-517', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Thank you for your approval. Is there something I can help you with or would you like to have a conversation?', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1738947178, model='llama3.2', object='chat.completion', service_tier=None, system_fingerprint='fp_ollama', usage=CompletionUsage(completion_tokens=24, prompt_tokens=28, total_tokens=52, completion_tokens_details=None, prompt_tokens_details=None))
Thank you for your approval. Is there something I can help you with or would you like to have a conversation?


# Website Summarizer

In [17]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [22]:
texts = Website("https://edwarddonner.com")

In [23]:
texts.title, texts.text

('Home - Edward Donner',
 'Home\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,\nacquired in 2021\n.\nWe work with groundbreaking, proprietary LLMs verticalized for talent, we’ve\npatented\nour matching model, and our award-winning platform has happy customers and tons of press cover

# Writing prompt

In [24]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."


def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt



In [26]:
print(user_prompt_for(text))

You are looking at a website titled Home - Edward Donner
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

Home
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.

In [29]:
def messages_for(webiste_text):
    return [
       { "role": "system", "content": system_prompt},
       {"role": "user", "content": user_prompt_for(webiste_text)}
        
    ]
    
    

In [30]:
messages_for(texts)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': 'You are looking at a website titled Home - Edward Donner\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nHome\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. We’re applying AI to a field where it can make a massive, posi

# All in one

In [33]:
def summarize(url):
    website_text = Website(url)
    response = openai.chat.completions.create(model = MODEL, messages= messages_for(website_text))
    return response.choices[0].message.content

In [34]:
summarize("https://edwarddonner.com")

'**Summary of Edward Donner\'s Website**\n=====================================\n\n### Overview\n\nThis website, `Home - Edward Donner`, provides an introduction to the work and interests of its creator, Ed. The site is primarily a personal blog, where Ed shares updates about his professional endeavors, hobbies, and connections with other individuals.\n\n### News and Announcements\n\n*   **December 21, 2024**: "LLM Workshop – Hands-on with Agents - resources" \n    *   This is a resource for LLM (Large Language Model) workshop attendees.\n*   **January 23, 2025**: No update available yet\n*   **November 13, 2024**: "Welcome, SuperDataScientists!" \n    This is an advertisement or welcome message to the SuperDataScientists community though details about this group are not available.\n\n### Features\n\nThe website features several sections:\n\n*   **About**\n    *   Introduces Ed\'s professional background and work with Nebula.io.\n    *   Mentions his previous endeavors at AI startup ut

In [40]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [41]:
display_summary("https://edwarddonner.com")

# Website Summary
### About the Creator
Edward Donner is a co-founder and CTO of Nebula.io, applying AI to help people discover their potential. He has experience as founder and CEO of AI startup untapt.

### Recent News/Announcements
#### December 2024
* Welcome, SuperDataScientists!: A welcome message for the community.

#### October 2024
* From Software Engineer to AI Data Scientist – resources: Resources available for transitioning from software engineer to AI data scientist role.

#### November 2024
* Mastering AI and LLM Engineering – Resources: Resources available for mastering AI and LLM engineering.

#### December 2024
* LLM Workshop – Hands-on with Agents – resources: Resources available for participating in the LLM workshop on agents.

### Website Content
The website primarily serves as an arena, Outsmart, to pit Large Language Models (LLMs) against each other in a competition. It also features updates about the creator's work with Nebula.io and the latest news/announcements from the field of AI and LLMs.

In [42]:
display_summary("https://google.com")

### Google Website Summary

#### Main Content

* Offers various services, including:
	+ Gmail
	+ Images (Google Photos)
* Provides localized versions of its platform in multiple languages, including Hindi, Bangla, Telugu, Marathi, Tamil, Gujarati, and more.

#### Notable News or Announcements

* None mentioned.
* However, the presence of a "Dark theme: Off" button suggests that Google may be considering or updating its user interface preferences.

In [ ]:
# To Do:
# Summarizer with Questions asked 